In [1]:
### PACKAGES
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os

In [2]:
#functions

def timestampTotime(df):
    timestamps = df['timestamp']
    folder = []
    folder_full = []
    for stamp in timestamps:
        time = datetime.fromtimestamp(stamp)
        #strtime = time.strftime('%H:%M:%S')
        strtime = time.strftime('%y/%m/%d-%H:%M:%S')
        strtimee = datetime.strptime(strtime, '%y/%m/%d-%H:%M:%S')
        folder.append(strtime)
        folder_full.append(strtimee)
    return folder, folder_full

def baseline_correction_val(df, t_start, t_stop):
    delta_t = 2
    idx_start = df[df.date == t_start].index[0]
    print(idx_start)
    #print(df['current'].iloc[idx_start])
    print(df['current'].loc[idx_start])
    idx_stop = df[df.date == t_stop].index[0]
    print(idx_stop)
    print(df.loc[idx_start-1:idx_stop]['current'])
    charge = sum(df.loc[idx_start-1:idx_stop]['current'])
    mean_current = charge/len(df['date'].loc[idx_start-1:idx_stop])
    return mean_current  

# corrected current
def baselineShift(df, start, end):
    #mean_current = baseline_correction_val(df,start,end)
    mean_current = 0.004121095820849825
    df.iloc[:, 1] -= mean_current
    return mean_current

def sep_caen1_caen2(directory):
    path_list_1 = []
    path_list_2 = []
    delta = timedelta(days = 1)
    today = datetime.today()
    
    try:
        # List all files in the specified directory
        files = os.listdir(directory)
        #print(files)
        
        # Check each file name
        for file in files:
            today_str = today.strftime('%Y-%m-%d')
            if "tem" in file:
                continue
            if "~" in file:
                continue
            if "save" in file:
                continue
            if "caen1" in file:
                path_list_1.append(os.path.abspath(directory + file))
            if "caen2" in file:
                path_list_2.append(os.path.abspath(directory + file))

    except FileNotFoundError:
        print(f"The directory {directory} was not found.")
        return False
    except PermissionError:
        print(f"Permission denied to access the directory {directory}.")
        return False
    
    return path_list_1, path_list_2

def total_charge(df, t_start, t_end):
    time, date = timestampTotime(df) # we have the date also in a list in case we need it
    current = df.IMON
    delta_t = 2
    voltage = df.VMON
    charge = 0
    #print(voltage)
    data = {'date': time,'current': current, 'voltage': voltage}
    new_df = pd.DataFrame(data)
    print(new_df.iloc[0:,1])
    mean_current = baselineShift(new_df, t_start, t_end)
    print(new_df)
    print(sum(new_df.iloc[:,1]))
    print(f'current used to baseline correction is {mean_current}')
    for i in range(0,len(new_df['voltage'])-1):
        if abs(new_df['voltage'].iloc[i+1] - new_df['voltage'].iloc[i]) < 1.0:
            #print(new_df[new_df.voltage == new_df['voltage'].iloc[i]].index[0])
            charge += new_df['current'].iloc[i]*delta_t
            #print(charge)
        else:
            continue
    return charge, mean_current

In [ ]:
directory = '/home/caio/data/240820_4xFEA1Test/datalogs/'
path_list_1, path_list_2 = sep_caen1_caen2(directory)

path_list_1.sort()
path_list_2.sort()

print(path_list_1)
df1 = [pd.read_csv(files, sep=',') for files in path_list_1]
df2 = [pd.read_csv(files, sep=',') for files in path_list_2]

df_caen1 = pd.concat(df1, ignore_index=True)
df_caen2 = pd.concat(df2, ignore_index=True)

In [36]:
# caen1 df per channel
df_caen1HV0 = df_caen1.loc[df_caen1["ch"] == 0]
df_caen1HV1 = df_caen1.loc[df_caen1['ch'] == 1]
df_caen1HV2 = df_caen1.loc[df_caen1["ch"] == 2]
df_caen1HV3 = df_caen1.loc[df_caen1['ch'] == 3]

# caen2 df per channel
df_caen2HV0 = df_caen2.loc[df_caen2["ch"] == 0]
df_caen2HV1 = df_caen2.loc[df_caen2['ch'] == 1]
df_caen2HV2 = df_caen2.loc[df_caen2["ch"] == 2]
df_caen2HV3 = df_caen2.loc[df_caen2['ch'] == 3]

###############################################################################


In [ ]:
t_start = '24/08/20-00:00:01'
t_end = '24/08/20-07:00:01'

df = df_caen1HV0

time, date = timestampTotime(df) # we have the date also in a list in case we need it
current = df.IMON
delta_t = 2
voltage = df.VMON
charge = 0
#print(voltage)
data = {'date': time,'current': current, 'voltage': voltage}
new_df = pd.DataFrame(data)

baseline_correction_val(new_df, t_start, t_end)

In [ ]:
t_start = '24/08/27-20:00:01'
t_end = '24/08/28-10:00:02'

total_charge(df_caen1HV0, t_start, t_end)

In [ ]:
delta_t = 2
today = datetime.today()

# period to take as reference to calculate the correction current
t_start = '24/08/27-20:00:01'
t_end = '24/08/28-10:00:02'

t_start2 = '24/08/27-20:00:02'
t_end2 = '24/08/28-10:00:02'

# 
all_df_caen1 = [df_caen1HV0, df_caen1HV1, df_caen1HV2, df_caen1HV3]
all_df_caen2 = [df_caen2HV0, df_caen2HV1, df_caen2HV2, df_caen2HV3]

result_caen1 = [total_charge(all_df_caen1[0], t_start, t_end), total_charge(all_df_caen1[1], t_start, t_end), total_charge(all_df_caen1[2], t_start, t_end), total_charge(all_df_caen1[3], t_start, t_end)]
result_caen2 = [total_charge(all_df_caen2[0], t_start2, t_end2), total_charge(all_df_caen2[1], t_start2, t_end2), total_charge(all_df_caen2[2], t_start2, t_end2), total_charge(all_df_caen2[3], t_start2, t_end2)]


charge_1 = [result_caen1[i][0] for i in range(0, len(all_df_caen1))]
charge_2 = [result_caen2[i][0] for i in range(0, len(all_df_caen2))]

mean_current_1 = [result_caen1[i][1] for i in range(0, len(all_df_caen1))]
mean_current_2 = [result_caen2[i][1] for i in range(0, len(all_df_caen2))]


print(f'\ncharge of caen1 HV0 is {charge_1[0]} uC' 
      f'\ncharge of caen1 HV1 is {charge_1[1]} uC' 
      f'\ncharge of caen1 HV2 is {charge_1[2]} uC' 
      f'\ncharge of caen1 HV3 is {charge_1[3]} uC\n'
      f'\ncharge of caen2 HV0 is {charge_2[0]} uC' 
      f'\ncharge of caen2 HV1 is {charge_2[1]} uC' 
      f'\ncharge of caen2 HV2 is {charge_2[2]} uC' 
      f'\ncharge of caen2 HV3 is {charge_2[3]} uC\n'
      
      f'last updated {today}\n')

In [ ]:
charge_1 = [result_caen1[i][0] for i in range(0, len(all_df_caen1))]
charge_2 = [result_caen2[i][0] for i in range(0, len(all_df_caen2))]

print(charge_1)


In [ ]:
t_start = '24/08/20-00:00:01'
t_end = '24/08/20-07:00:01'
df = df_caen1HV2
charge = total_charge(df, t_start, t_end)

In [ ]:
msg = {"from": "caen1_132143", "timestamp": 1724307611.4407609, "payload": [{"tags": {"channel": 0}, "data": {"VMON": 1.85, "VSET": 40.0, "VMAX": 6000.0, "IMRANGE": "HIGH", "IMON": 0.0044, "ISET": 10.0, "POL": "+", "RUP": 50.0, "RDW": 50.0, "TRIP": 10.0, "POWER": "off"}}, {"tags": {"channel": 1}, "data": {"VMON": 1.45, "VSET": 20.0, "VMAX": 6000.0, "IMRANGE": "HIGH", "IMON": 0.0038, "ISET": 2.1, "POL": "+", "RUP": 50.0, "RDW": 50.0, "TRIP": 10.0, "POWER": "off"}}, {"tags": {"channel": 2}, "data": {"VMON": 1.2, "VSET": 0.0, "VMAX": 6000.0, "IMRANGE": "LOW", "IMON": -0.00511, "ISET": 2.1, "POL": "-", "RUP": 2.0, "RDW": 50.0, "TRIP": 10.0, "POWER": "off"}}, {"tags": {"channel": 3}, "data": {"VMON": 1410.15, "VSET": 1410.0, "VMAX": 6000.0, "IMRANGE": "LOW", "IMON": 0.01249, "ISET": 2.1, "POL": "-", "RUP": 2.0, "RDW": 50.0, "TRIP": 10.0, "POWER": "on"}}]}

msg['payload'][0]['data']['IMON']

In [ ]:
for l in msg['payload']:
        #source =  get_source_from_topic(topic)
        #print(l['tags']['channel'])
        current = l['data']['IMON']
        #print(current)
        ramp_up = l['data']['RAMPINGUP']
        ramp_down = l['data']['RAMPINGDOWN']
        channel = l['tags']['channel']
        if current >= 0:
                if ramp_up == 'no' and ramp_down == 'no':
                        integration[channel] += (current - mean_current[channel])*delta_t
        sc = msg['from']
        integration_val = integration[channel]
        print(f'charge of {sc} {channel} is {integration_val} uC')